<a href="https://colab.research.google.com/github/theFishy-1/OCR/blob/OCR/tesseractv1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pybind11
!pip install fastwer
!pip install pytesseract
!sudo apt install tesseract-ocr
!pip install pytesseract pillow pdf2image
!apt-get update
!apt-get install -y poppler-utils tesseract-ocr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.3/243.3 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fastwer: filename=fastwer-0.1.3-cp311-cp311-linux_x86_64.whl size=866384 sha256=460e07f1bd90d5117e06aa73ae9cfe4ae411ca3d4310b35b487ff1c1b5942959
  Stored in directory: /root/.cache/pip/wheels/4c/53/1e/8d806da8c1ed1de60e371005658af32b92aad9426b37208f1f
Successfully built fastwer
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 23 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 htt

In [2]:
import cv2
import pytesseract
import fastwer
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import Image, display
from IPython import get_ipython
from PIL import Image
from pdf2image import convert_from_path


In [11]:
folder_path = "./sample_images"
files = sorted([f for f in os.listdir(folder_path) if f.endswith(('.png', '.jpg', '.jpeg', '.pdf'))])

df_output = pd.DataFrame(columns=['img_filename', 'ocr_output'])


custom_config = r'--oem 3 --psm 6 -l pol'

for file in files:
    file_path = os.path.join(folder_path, file)


    if file.endswith(('.png', '.jpg', '.jpeg')):
        im = cv2.imread(file_path)
        if im is None:
            print(f"Błąd wczytywania obrazu: {file}")
            continue
        img_rgb = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
        output = pytesseract.image_to_string(img_rgb, config=custom_config)


    elif file.endswith('.pdf'):
        try:
            images = convert_from_path(file_path)  # Konwersja PDF na obrazy
            output = ""
            for i, img in enumerate(images):
                text = pytesseract.image_to_string(img, config=custom_config)
                output += f"=== Strona {i+1} ===\n{text}\n"
        except Exception as e:
            print(f"Błąd przetwarzania PDF: {file}, {e}")
            continue

    print(f"OCR dla {file}:\n{output}\n{'-'*50}")


    dictionary = {'img_filename': file, 'ocr_output': str(output)}
    df_output = pd.concat([df_output, pd.DataFrame([dictionary])], ignore_index=True)


    output_filename = os.path.join(folder_path, f"{file}.txt")
    with open(output_filename, "w", encoding="utf-8") as text_file:
        text_file.write(output)

OCR dla PDF1.pdf:
=== Strona 1 ===
gia Warszawa, dnia 16 kwietnia 2021 r.
Zjem
RZECZPOSPOLITA POLSKA
MINISTER FINANSÓW,
FUNDUSZY I POLITYKI
REGIONALNEJ

DCT2.054.1.2021DCT2.054.1.2021

Pani

Elżbieta Witek

Marszałek Sejmu

Rzeczypospolitej Polskiej
Szanowna Pani Marszałek,
w odpowiedzi na Interpelację nr 19562 Pana Posła Grzegorza Gaża w sprawie istnienia obowiązku
dokumentacyjnego w zakresie cen transferowych przekazuję poniższe stanowisko.
Ad. 1. Czy obowiązkowi dokumentacyjnemu podlegają także umowy przedwstępne, jeżeli
w roku podatkowym, za który sporządzana jest dokumentacja cen transferowych, nie doszło
do zawarcia umów przyrzeczonych?
Zgodnie z art. 11k ust. 2 ustawy z dnia 15 lutego 1992 r. o podatku dochodowym od osób
prawnych (Dz. U. z 2020 r. poz. 1406, z późn. zm., dalej: ustawa o CIT) oraz art. 23w ust. 2 ustawy
z dnia 26 lipca 1991 r. o podatku dochodowym od osób fizycznych (Dz. U. z 2020 r. poz. 1426,
z późn. zm., dalej: ustawa o PIT) lokalna dokumentacja cen transferow

In [14]:
# Create new columns for reference, CER and WER
df_output['ref_text'] = ''
df_output['cer'] = ''
df_output['wer'] = ''

# Replace new lines in output
df_output['ocr_output'] = df_output['ocr_output'].apply(lambda x: x.replace('\n',''))

df_output.loc[df_output['img_filename'] == 'PNG1.png', 'ref_text'] = "Wrocław, 24.08.2022 Jan Kowalski ul. Piękna 1 50-100 Wrocław tel. 600 100 200 e-mail. j.kowalski@gmail.com Prezes zarządu Anna Nowak Przedsiębiorstwo Pol-Bud Sp. z o.o. ul. Ładna 1 56-400 Oleśnica PODANIE Zwracam się z uprzejmą prośbą o przedłużenie dotychczasowej umowy o pracę na czas określony oraz o zwiększenie stawki wynagrodzenia. Swoją prośbę motywuję zwiększeniem moich kwalifikacji tj. uzyskaniem dyplomu inżyniera budownictwa. Chciałbym również nadmienić, że w dotychczasowym okresie zatrudnienia, moja praca była pozytywnie oceniana przez mojego przełożonego, kierownika działu zaopatrzenia p. Marka Kwiatkowskiego. Będę bardzo wdzięczny za pozytywne rozpatrzenie mojej prośby. Z wyrazami szacunku Jan Kowalski Załączniki: Załącznik nr 1 - kserokopia dyplomu inżyniera budownictwa Załącznik nr 2-opinia kierownika działu zaopatrzenia"
df_output.loc[df_output['img_filename'] == 'PNG2.png', 'ref_text'] = "Warszawa, dn. 09 sierpnia 2016 r MINISTER OBRONY NARODOWEJ 2016-08-09 RUCHU OBYWATELSKIEGO MIŁOŚNIKÓW BRONI Pan Andrzej TURCZYN ul. Chłodna 20/109 00-891 Warszawa Dotyczy: zmiany zasad dostępu do broni palnej. W odpowiedzi na pismo z dnia 28 czerwca 2016 r., w sprawie zmiany zasad dostępu do broni palnej, uprzejmie informuję, że w ramach prac związanych z procesem tworzenia Wojsk Obrony Terytorialnej, trwają również prace nad stworzeniem systemu przechowywania i użytkowania broni palnej przez żołnierzy obrony terytorialnej. Aktualnie rozwazane są dwa warianty przechowywania broni palnej Pierwszy z nich przewiduje przechowywanie broni i sprzętu wojskowego w aktualnie istniejących jednostkach wojskowych. W wariancie drugim, w perspektywie dwóch lat, planuje się aby część broni osobistej przechowywać wykorzystując w tym celu wolne miejsca w magazynach broni Policji oraz innych służb mundurowych, na terenie powiatów. Proponowany przez Ruch Obywatelski Miłośników Broni wariant przechowywania broni palnej przez żołnierzy, w miejscu ich zamieszkania jest ciekawymi wartym rozważenia rozwiązaniem, jednak wymaga przeprowadzenia szerokich konsultacji społecznych oraz gruntownych zmian prawnych organizacyjnych w Silach Zbrojnych RP, z tego powodu będzie możliwy do zastosowania w WOT najwcześniej za kilka lat. W przypadku podjęcia decyzji o rozpoczęciu prac legislacyjnych przewidujących praktyczną realizację trzeciego wariantu, stosowny projekt zostanie przesłany w ramach konsultacji do Ruchu Obywatelskiego Miłośników Brani. Przedstawiając powyższe, pragnę podziękować Panu Prezesowi za zainteresowanie omawianymi kwestiami. wz. Bartosz KOWNACKi"
df_output.loc[df_output['img_filename'] == 'PNG3.jpg', 'ref_text'] = "RZECZPOSPOLITA POLSKA MINISTERSTWO FINANSÓW SEKRETARZ STANU Artur Soboń Warszawa, 8 lipca 2022 roku Sprawa: Odpowiedź na pismo - minimalny podatek dochodowy - podmioty lecznicze Znak sprawy: DD6.8223.14.2022 Kontakt: Kancelaria MF tel. +48 22 694 55 55 e-mail: kancelaria@mf.gov.pl Pan Władysław Perchaluk Prezes Związku Szpitali Powiatowych Województwa Śląskiego ul. Żeromskiego 7 41-902 Bytom biuro@szpitalepowiatowe.pl Szanowny Panie Prezesie, w związku z wystąpieniem skierowanym 7 lutego 2022 r.1, zawierającym postulat nowelizacji ustawy CIT2, celem wyłączenia z opodatkowania minimalnym podatkiem dochodowym (minimalny CIT) podmiotów leczniczych udzielających świadczeń opieki zdrowotnej finansowanych ze środków publicznych, proszę przyjąć następujące wyjaśnienia. 1 stycznia 2022 r. wszedł w życie art. 24ca ustawy CIT wprowadzający opodatkowanie minimalnym CIT. Mając na uwadze wpływające do Ministerstwa Finansów zastrzeżenia - podnoszone zarówno przez organizacje przedsiębiorców skupiające poszczególne branże, doradców podatkowych, jak i samych podatników a także uwzględniając sytuację geopolityczną, wynikającą z czynników zewnętrznych, podjęta została decyzja o konieczności przeprowadzenia weryfikacji założeń, które legły u podstaw konstrukcji tego podatku. Założeniami jakie przyświecały wspomnianym analizom były: korekta przepisów budzących wątpliwości interpretacyjne lub prowadzących do rezultatów sprzecznych z ideą regulacji o minimalnym CIT, większe uwzględnienie specyfiki poszczególnych branż, 1 przekazanym Ministerstwu Finansów przy piśmie Biura Prezesa Kancelarii Prezesa Rady Ministrów z 15 lutego 2022 r. znak BPRM.5032.1.21.2022(2)JG 2 Ustawa z 15 lutego 1992 r. o podatku dochodowym od osób prawnych (Dz. U. z 2021 r. poz. 1800, z późn.zm.). Ministerstwo Finansów ul. Świętokrzyska 12 00-916 Warszawa tel.: +48 22 694 55 55 fax: +48 22 694 36 84 e-mail: kancelaria@mf.gov.pl"
df_output.loc[df_output['img_filename'] == 'PDF1.pdf', 'ref_text'] = "Warszawa, dnia 16 kwietnia 2021 r. RZECZPOSPOLITA POLSKA MINISTER FINANSÓW, FUNDUSZY I POLITYKI REGIONALNEJ DCT2.054.1.2021DCT2.054.1.2021 Pani Elżbieta Witek Marszałek Sejmu Rzeczypospolitej Polskiej Szanowna Pani Marszałek, w odpowiedzi na Interpelację nr 19562 Pana Posła Grzegorza Gaża w sprawie istnienia obowiązku dokumentacyjnego w zakresie cen transferowych przekazuję poniższe stanowisko. Ad. 1. Czy obowiązkowi dokumentacyjnemu podlegają także umowy przedwstępne, jeżeli w roku podatkowym, za który sporządzana jest dokumentacja cen transferowych, nie doszło do zawarcia umów przyrzeczonych? Zgodnie z art. 11k ust. 2 ustawy z dnia 15 lutego 1992 r. o podatku dochodowym od osób prawnych (Dz. U. z 2020 r. poz. 1406, z późn. zm., dalej: ustawa o CIT) oraz art. 23w ust. 2 ustawy z dnia 26 lipca 1991 r. o podatku dochodowym od osób fizycznych (Dz. U. z 2020 r. poz. 1426, z późn. zm., dalej: ustawa o PIT) lokalna dokumentacja cen transferowych jest sporządzana dla „transakcji kontrolowanej”. A zatem należy zbadać, czy zawarcie umowy przedwstępnej w kontekście obowiązku sporządzenia dokumentacji cen transferowych mieści się w definicji transakcji kontrolowanej. Pojęcie transakcji kontrolowanej zostało zdefiniowane w art. 11a ust. 1 pkt 6 ustawy o CIT (art. 23m ust. 1 pkt 6 ustawy o PIT) i oznacza identyfikowanie na podstawie rzeczywistych zachowań stron, działania o charakterze gospodarczym, w tym przypisywanie dochodów do ul. Świętokrzyska 12, 00-916 Warszawa  |  tel.: +48 (22) 694 55 55  |  e-mail: kancelaria@mf.gov.pl zagranicznego zakładu, których warunki zostały ustalone lub narzucone w wyniku powiązań. Oznacza to, że podstawowym elementem definiującym transakcję kontrolowaną pomiędzy podmiotami powiązanymi jest jej charakter gospodarczy. Umowa przedwstępna nie należy do kategorii umów wzajemnych: „(…) oświadczenia woli zawarte w umowie przedwstępnej nie da się ocenić w kategoriach ekonomicznych i nie mogą być przedmiotem obrotu” (Kodeks Cywilny T. I. Komentarz do art. 1-449(10), red. Pietrzykowski, 2021, wyd. 10) oraz „Zobowiązanie do złożenia w przyszłości określonego oświadczenia woli nie stanowi bowiem ekwiwalentu, zwłaszcza ekonomicznego (odpłaty), analogicznego zobowiązania drugiej strony” (Tom III A. Kodeks cywilny. Komentarz. Zobowiązania. Część ogólna, red. Osajda, 2021, wyd. 27). Podobne stanowisko prezentowane jest w orzecznictwie Sądu Najwyższego: „(…) umowa przedwstępna nie jest umową wzajemną, gdyż stanowiąc tylko instrument prawny prowadzący do zawarcia umowy przyrzeczonej reguluje interesy stron na ”przedpolu” zawarcia umowy przyrzeczonej.” (wyrok Sądu Najwyższego z dnia 25 lutego 2016 r. III CSK 136/15) oraz „Umowa przedwstępna, jako umowa przygotowawcza, tym różni się od umów definitywnych, że nie stanowi źródła obowiązku świadczenia, które bezpośrednio realizuje zamierzony przez strony cel gospodarczy” (wyrok Sądu Najwyższego z dnia 11 marca 2016 r. I CSK 161/15). W związku z powyższym, zdaniem Ministra Finansów, Funduszy i Polityki Regionalnej,  umowa przedwstępna oraz wynikające z niej skutki prawne nie spełniają, ze względu na swoją charakterystykę, przesłanek definicji transakcji kontrolowanej. Podstawową cechą transakcji kontrolowanej jest jej charakter gospodarczy, czego nie można powiedzieć o umowie przedwstępnej stanowiącej jedynie zobowiązanie do zawarcia określonej umowy w przyszłości. Nie powstaje przy tym stosunek odpłatny charakterystyczny dla umów wzajemnych. Umowy przedwstępne występujące w obrocie gospodarczym mogą również zawierać postanowienia o prawie do odstąpienia przed zawarciem umowy przyrzeczonej (tzw. umowne prawo do odstąpienia). Oznacza to bezcelowość sporządzania dokumentacji cen transferowych dla umów, które w łatwy sposób mogą zostać rozwiązane na podstawie umownych przesłanek ustalonych przez strony. Również wartość transakcji kontrolowanej (warunkująca powstanie obowiązku dokumentacyjnego po przekroczeniu progów ustawowych) nie powinna być określana na podstawie postanowień umowy przedwstępnej. Jedynie zawarcie umowy przyrzeczonej mogłoby stanowić podstawę do określenia wartości transakcji kontrolowanej zgodnie z art. 11l ustawy o CIT (art. 23x ustawy o PIT). Podsumowując, obowiązkowi dokumentacyjnemu wynikającemu z przepisów o cenach transferowych nie podlegają umowy przedwstępne, w szczególności w przypadku gdy w roku podatkowym, za który sporządzana jest dokumentacja cen transferowych, nie doszło do zawarcia umów przyrzeczonych. Ad. 2. W jaki sposób rozumieć zawarty w art. 11o ust. 1a ustawy o podatku dochodowym od osób prawnych limit 500 000 zł? Czy limit ten odnosi się do transakcji z jednym rzeczywistym właścicielem, czy też należy w jego ramach sumować wartości transakcji o charakterze jednorodnym, zawieranych z różnymi kontrahentami przypisywanymi różnym rzeczywistym właścicielom, jeżeli rzeczywisty właściciel ma miejsce zamieszkania siedzibę lub zarząd na terytorium lub w kraju stosującym szkodliwą konkurencję podatkową? Zgodnie z art. 11o ust. 1a ustawy o CIT oraz art. 23za ust. 1a ustawy o PIT do sporządzenia lokalnej dokumentacji cen transferowych są obowiązani podatnicy i spółki niebędące osobami prawnymi dokonujący transakcji kontrolowanej lub transakcji innej niż transakcja kontrolowana, jeżeli rzeczywisty właściciel ma miejsce zamieszkania, siedzibę lub zarząd na terytorium lub w kraju stosującym szkodliwą konkurencję podatkową (tzw. raju podatkowym) oraz wartość tej transakcji za rok podatkowy, a w przypadku spółek niebędących osobami prawnymi – za rok obrotowy, przekracza 500 000 zł. Powołany przepis określa dwa warunki, które łącznie spełnione zobowiązują do sporządzenia dokumentacji cen transferowych:  przekroczenie limitu wynoszącego 500 000 zł dla transakcji,  rzeczywisty właściciel jest podmiotem mającym miejsce zamieszkania, siedzibę lub zarząd w tzw. raju podatkowym. Na potrzeby powołanej regulacji w art. 11o ust. 1b ustawy o CIT (art. 23za ust. 1b ustawy o PIT) ustanowiono domniemanie, że rzeczywisty właściciel posiada miejsce zamieszkania, siedzibę lub zarząd w tzw. raju podatkowym, w sytuacji gdy kontrahent podatnika lub spółki niebędącej osobą prawną dokonuje w roku podatkowym rozliczeń z tzw. podmiotem rajowym. Domniemanie służy uproszczeniu przy dokonywaniu oceny w zakresie rzeczywistego właściciela oraz jego siedziby, ponieważ ustalenie tych okoliczności może okazać się w wielu przypadkach obiektywnie skomplikowane. Zdaniem Ministra Finansów, Funduszy i Polityki Regionalnej, obowiązku dokumentacyjnego wynikającego z art. 11o ust. 1a ustawy o CIT (art. 23za ust. 1a ustawy o PIT) nie należy analizować w oderwaniu od ust. 1 tego artykułu, który ustanawia obowiązek dokumentacji transakcji innych niż transakcje kontrolowane zawieranych „z podmiotem mającym miejsce zamieszkania, siedzibę lub zarząd” w tzw. raju podatkowym, których wartość za rok podatkowy (obrotowy) przekracza 100 000 zł. Odesłanie w art. 11o ust. 1 ustawy o CIT (art. 23za ust. 1 ustawy o PIT) do odpowiedniego zastosowania art. 11k ust. 3-5 ww. ustawy (art. 23w ust. 3-5 ustawy o PIT) nakazuje, aby wartość transakcji właściwą dla ustalenia, czy został przekroczony próg kwotowy, określać dla transakcji o charakterze jednorodnym. Zgodnie z art. 11k ust. 4 ustawy o CIT (art. 23w ust. 4 ustawy o PIT) wartość transakcji o charakterze jednorodnym jest ustalana bez względu na:  liczbę dokumentów księgowych,  dokonanych lub otrzymanych płatności oraz  podmiotów powiązanych, z którymi zawierana jest transakcja kontrolowana. Zdaniem Ministra Finansów, Funduszy i Polityki Regionalnej, tylko dwa pierwsze kryteria mają wpływ na ocenę jednorodności transakcji innej niż transakcja kontrolowana z podmiotem z tzw. rajów podatkowych. Natomiast kryterium „liczby podmiotów powiązanych, z którymi zawierana jest transakcja kontrolowana” wskazane w art. 11k ust. 4 ustawy o CIT (art. 23w ust. 4 ustawy o PIT) nie znajduje zastosowania, mając na uwadze, że treść ust. 1 art. 11o ustawy o CIT (ust. 1 art. 23za ustawy o PIT) wskazuje na transakcje zawierane z jednym podmiotem. Oznacza to, że przy ustaleniu jednorodności transakcji na gruncie art. 11o ust. 1 ustawy o CIT (art. 23za ust. 1 ustawy o PIT) należy wziąć pod uwagę relacje z jednym podmiotem mającym miejsce zamieszkania, siedzibę lub zarząd w tzw. raju podatkowym. Zakres odpowiedniego stosowania art. 11k ust. 4 ustawy o CIT (art. 23w ust. 4 ustawy o PIT) do transakcji innych niż transakcje kontrolowane, przewidziany w art. 11o ust. 1a ustawy o CIT (art. 23za ust. 1a ustawy o PIT), należy odczytywać w powiązaniu z przepisem ust. 1 tego artykułu. Odesłanie nie dotyczy zatem tej części tego przepisu mówiącej o obowiązku ustalania wartości transakcji bez względu na liczbę „(…) podmiotów powiązanych, z którymi zawierana jest transakcja kontrolowana”. Ustalając wartość jednorodnej transakcji innej niż transakcja kontrolowana, jeżeli rzeczywisty właściciel jest rezydentem w tzw. raju podatkowym, należy brać pod uwagę transakcje zawarte z jednym kontrahentem, które spełniają pozostałe kryteria jednorodności wskazane w art. 11k ust. 4 ustawy o CIT (art. 23w ust. 4 ustawy o PIT). Wnioskowanie takie wynika z przewidzianego w art. 11o ust. 1a ustawy o CIT (art. 23za ust. 1a ustawy o PIT) odpowiedniego stosowania przepisów. Odpowiednie stosowanie przepisów może oznaczać stosowanie ich wprost, w całości, stosowanie z modyfikacjami wynikającymi z istoty regulowanej instytucji prawnej, względnie oznacza odmowę ich stosowania. Wskazane w powołanym ustępie przepisy należy stosować przy uwzględnieniu modyfikacji wynikających z istoty regulowanej instytucji prawnej. Kryteria jednorodności wynikające z art. 11k ust. 4 ustawy o CIT (art. 23w ust. 4 ustawy o PIT), w przypadku transakcji innej niż transakcja kontrolowana, powinny być stosowane wyłącznie w zakresie właściwym dla relacji pomiędzy podmiotami niepowiązanymi. Oznacza to, że w tym przypadku kryterium jednorodności wyrażone w art. 11k ust. 4 ustawy o CIT in fine (art. 23w ust. 4 ustawy o PIT) nie znajdzie zastosowania. Podsumowując, dla transakcji innych niż transakcje kontrolowane, jeżeli rzeczywisty właściciel jest rezydentem w tzw. raju podatkowym, przewidziany w art. 11o ust. 1a ustawy o CIT (art. 23za ust. 1a ustawy o PIT) limit dokumentacyjny w wysokości 500 000 zł ustala się na podstawie transakcji o charakterze jednorodnym zawartej z jednym kontrahentem. Z poważaniem Podsekretarz Stanu z upoważnienia Ministra Finansów, Funduszy i Polityki Regionalnej Jan Sarnowski Podsekretarz Stanu w Ministerstwie Finansów Jan Sarnowski (podpisano kwalifikowanym podpisem elektronicznym)"
df_output.loc[df_output['img_filename'] == 'PDF2.pdf', 'ref_text'] = "RZECZPOSPOLITA POLSKA MINISTER FINANSÓW Warszawa, 15 lutego 2022 roku Sprawa: Pismo RPO w sprawie rozważenia możliwości podjęcia inicjatywy ustawodawczej w celu uznania okresów pracy w Wywiadzie Skarbowym oraz Wydziałach Realizacji UKS, jako równorzędnych ze służbą, na zasadach zbliżonych do funkcjonariuszy celnych i Straży Marszałkowskiej Znak sprawy: DWR1.055.1.2022 Kontakt: Kancelaria MF tel. +48 22 694 55 55 e-mail: kancelaria@mf.gov.pl Marcin Wiącek Rzecznik Praw Obywatelskich Odpowiedź Ministra Finansów Szanowny Panie Rzeczniku, w odpowiedzi na pismo Pana Rzecznika nr WZF.7060.91.2021.TO z 13.01.2022 r. w sprawie rozważenia możliwości podjęcia inicjatywy ustawodawczej w celu uznania okresów pracy w Wywiadzie Skarbowym oraz Wydziałach Realizacji UKS, jako równorzędnych ze służbą, na zasadach zbliżonych do funkcjonariuszy celnych i Straży Marszałkowskiej, przedstawiam następujące wyjaśnienia. W piśmie Pana Rzecznika wskazano, że gdyby ustawodawca literalnie zrealizował wyrok Trybunału Konstytucyjnego z 3 marca 2015 r. (sygn. akt K 39/13) w sprawie zaopatrzenia emerytalno-rentowego funkcjonariuszy celnych to świadczenia emerytalne powinny dotyczyć jedynie tych funkcjonariuszy Służby Celnej, którzy faktycznie realizowali tzw. „zadania policyjne”, i to jedynie za okres, w którym je rzeczywiście wykonywali. Tymczasem ustawodawca rozszerzył te uprawnienia praktycznie na wszystkich funkcjonariuszy byłej Służby Celnej, wprowadzając jedynie wymóg pięcioletniego okresu wykonywania „zadań policyjnych” w ramach służby w byłej Służbie Celnej lub pięcioletniej służby w Służbie Celno- Skarbowej. Rozwiązanie takie było korzystne dla funkcjonariuszy, bowiem zakres zadań konkretnego funkcjonariusza mógł ulegać zmianie na przestrzeni wskazanych lat. W tym zakresie należy także zauważyć, że w odróżnieniu od pozostałych pragmatyk służb mundurowych, w Służbie Celnej istniała możliwość dość płynnej zmiany formy zatrudnienia (za zgodą zainteresowanego, art. 98 i 99 ustawy o Służbie Celnej). Konsekwencją ul. Świętokrzyska 12   00-916 Warszawa 1/5 tel.: +48 22 694 55 55 • fax: +48 22 694 36 84 • e-mail: kancelaria@mf.gov.pl przekształcenia stosunku służbowego w stosunek pracy był brak możliwości wykonywania zadań zastrzeżonych wyłącznie dla funkcjonariuszy celnych. Z uwagi na fakt, że pracownik cywilny nie mógł wykonywać zadań o policyjnym charakterze, wysługę emerytalną stanowią wyłącznie okresy służby w Służbie Celnej i Służbie Celno- Skarbowej. Ponadto Pan Rzecznik stwierdza, iż sytuacja pracowników Wywiadu Skarbowego i Wydziałów Realizacji UKS jest inna, gdyż byli oni pracownikami wykonującymi zadania o policyjnym charakterze i to właśnie ten policyjny charakter jest wspólnym mianownikiem (stanowi cechę relewantną) przesądzającym o naruszeniu zasady równego traktowania funkcjonariuszy Służby Celno- Skarbowej (wywodzących się, przed konsolidacją, z różnych instytucji, ale działających w ramach jednego resortu). Odpowiadając na pismo Pana Rzecznika należy ponownie wyjaśnić, że objęcie funkcjonariuszy Służby Celno-Skarbowej i  funkcjonariuszy Służby Celnej zaopatrzeniem emerytalnym służb mundurowych było skutkiem m. in. treści wyroku Trybunału Konstytucyjnego z 3 marca 2015 r. (sygnatura akt K39/13) w sprawie uprawnień emerytalnych funkcjonariuszy Służby Celnej. W zakresie terminu 5 lat wykonywania zadań policyjnych należy wyjaśnić, iż zgodnie z treścią wyroku Trybunał nie wskazał, przez jaki okres funkcjonariusz powinien wykonywać wskazane zadania w okresie swojej służby, czy to ma być miesiąc, rok czy też cały okres służby. W wyroku nie określono w jaki sposób ustawodawca powinien zrealizować postanowienia wyroku. Z uwagi na powyższe oraz mając na względzie warunki pełnienia służby, w szczególności związane z dyspozycyjnością, podporządkowaniem, możliwością przenoszenia funkcjonariuszy z uwagi na potrzeby służby, ustawodawca realizując wyrok Trybunału określił ramy czasowe przy realizacji tzw. zadań policyjnych uprawniające do świadczeń emerytalnych służb mundurowych. Należy jeszcze raz podkreślić, iż  emerytura policyjna przysługuje funkcjonariuszowi z tytułu pełnienia służby w Policji, Agencji Bezpieczeństwa Wewnętrznego, Agencji Wywiadu, Służbie Kontrwywiadu Wojskowego, Służbie Wywiadu Wojskowego, Centralnym Biurze Antykorupcyjnym, Straży Granicznej, Straży Marszałkowskiej, Służbie Ochrony Państwa, Państwowej Straży Pożarnej, Służbie Celnej, Służbie Celno-Skarbowej lub Służbie Więziennej. Jako równorzędne ze służbą w Policji, ABW, AW, itd. traktuje się okresy służby w UOP, BOR, służbę wojskową, oraz wyjątkowo okresy zatrudnienia, o których mowa w art. 13 i art. 18b ustawy z dnia 18 lutego 1994 r. o zaopatrzeniu emerytalnym funkcjonariuszy Policji, Agencji Bezpieczeństwa Wewnętrznego, Agencji Wywiadu, Służby Kontrwywiadu Wojskowego, Służby Wywiadu Wojskowego, Centralnego Biura Antykorupcyjnego, Straży Granicznej, Straży Marszałkowskiej, Służby Ochrony Państwa, Państwowej Straży Pożarnej, Służby Celno- Skarbowej i Służby Więziennej oraz ich rodzin1. Zgodnie z art. 1 ustawy o zaopatrzeniu emerytalnym funkcjonariuszy Policji (…), zaopatrzenie emerytalne z tytułu wysługi lat przysługuje funkcjonariuszom Służby Celno-Skarbowej oraz w myśl ust. 2 również funkcjonariuszom Służby Celnej, którzy zostali mianowani do Służby Celnej po dniu 14 września 1999 r. lub których stosunek służbowy został przekształcony w stosunek służbowy na podstawie art. 22b ustawy z dnia 24 lipca 1999 r. o Służbie Celnej lub art. 99 ustawy z dnia 27 sierpnia 2009 r. o Służbie Celnej. Wysługę emerytalną stanowią okresy służby w Służbie Celnej i Służbie Celno-Skarbowej. Ponadto zgodnie z art. 13 ww. ustawy jako równorzędne ze służbą traktuje się okresy zatrudnienia w Służbie Celnej od dnia 15 września 1999 r. (od wejścia w życie ustawy z dnia 24 lipca 1999 r. o Służbie Celnej) do dnia przekształcenia stosunku pracy w stosunek służby w przypadku gdy osoba, która wykonywała zadania przypisane dla Służby Celnej, otrzymała akt mianowania skutkujący tym przekształceniem. Zgodnie z art. 91 powołanej ustawy o Służbie 1 Dz. U. z 2020 r. poz. 723 Celnej funkcjonariusze zatrudnieni w dniu wejścia w życie ustawy w administracji celnej, od tego dnia wykonywali zadania przewidziane dla Służby Celnej. W terminie 9 miesięcy od dnia wejścia w życie tej ustawy osoby te otrzymały pisemną propozycję służby. W myśl art. 92 ww. ustawy w przypadku przyjęcia propozycji pełnienia służby na stanowisku funkcjonariusza celnego, dotychczasowy stosunek pracy przekształcał się w stosunek służby z dniem mianowania. Mając na uwadze 9-miesięczny termin, który ustawodawca przewidział na wręczenie aktu mianowania, zasadne było aby okres zatrudnienia pomiędzy 15 września 1999 r. a dniem mianowania traktowany był na równi ze służbą dla tych funkcjonariuszy, którzy wykonywali zadania przewidziane dla Służby Celnej, a następnie otrzymali akty mianowania. Podsumowując, w przypadku funkcjonariuszy Służby Celnej i Służby Celno-Skarbowej do wysługi emerytalnej zaliczane są okresy służby. Datę wejścia w życie ustawy z dnia 24 lipca 1999 r. o Służbie Celnej, przypadającą na 15 września 1999 r. przyjęto jako datę początkową przy ustalaniu prawa do emerytury policyjnej i obliczania jej wysokości. W odniesieniu do Służby Celnej, nie uwzględniono okresów zatrudnienia funkcjonariuszy przed 1999 r., bowiem funkcjonariusze wykonujący pracę na podstawie ustawy z dnia 16 września 1982 r. o pracownikach państwowych nie stanowili służby mundurowej, lecz byli urzędnikami urzędów państwowych. W przedmiotowej sprawie należy mieć również na uwadze różnice związane ze stosunkiem pracy i stosunkiem służbowym. Stosunek służbowy funkcjonariuszy służb mundurowych nie jest stosunkiem pracy, lecz publicznoprawnym stosunkiem administracyjnym. Taki charakter stosunków służbowych funkcjonariuszy służb mundurowych charakteryzuje się znacznie większym, niż pracowników umownych, stopniem dyspozycyjności i hierarchicznego podporządkowania. Dyspozycyjność oraz podporządkowanie funkcjonariuszy związana jest z obowiązkami wykonywania przez funkcjonariuszy rozkazów, poleceń przełożonych, możliwością powierzania im pełnienia obowiązków służbowych na innym stanowisku oraz wyznaczania i przenoszenia na różne stanowiska, również w innej miejscowości, stosownie do potrzeb służby. W zamian za to ustawodawca gwarantuje funkcjonariuszom wzmożoną trwałość i stabilność zatrudnienia, przewidując, że stosunki służbowe mogą być modyfikowane tylko w przypadkach i na zasadach sprecyzowanych w poszczególnych ustawach służb mundurowych, a działania właściwych organów w tym zakresie co do zasady podlegają kontroli instancyjnej i sądowo administracyjnej. W systemie zaopatrzenia emerytalnego służb mundurowych o prawie do emerytury mundurowej/policyjnej decyduje określony czas służby i okresy równorzędne ze służbą zdefiniowane w art. 13 i art. 18b ustawy o zaopatrzeniu emerytalnym. Osoby urodzone po 31 grudnia 1948 r., które w czasie swojej aktywności zawodowej pozostawały zarówno w zatrudnieniu, jak i pełniły służbę - za okresy pracy w „cywilu” oraz okresy służby przez wymaganą liczbę lat (15/25 lat) mogą nabyć prawo do emerytury z dwóch systemów emerytalnych: powszechnego, po osiągnięciu ustawowego wieku emerytalnego i mundurowego, po spełnieniu wymaganego warunku stażu służby. Funkcjonariusz, który nie nabędzie prawa do emerytury mundurowej, gdyż nie spełni ustawowej przesłanki wymaganego stażu służby zostaje objęty ubezpieczeniem emerytalnym w systemie powszechnym, w którym nie ma warunku wymaganego stażu pracy – a prawo do emerytury z systemu powszechnego/ZUS nabywa każda osoba, która była ubezpieczona i zapłaciła chociażby tylko jedną składkę. W przypadku nie spełnienia przez funkcjonariusza warunków do nabycia prawa do emerytury mundurowej po zwolnieniu ze służby zostaną przekazane przez właściwy podmiot składki do ZUS za cały okres służby, zgodnie z art. 250a ustawy o KAS. W związku z powyższym, należy jeszcze raz podkreślić, iż byli pracownicy zatrudnieni w Wywiadzie Skarbowym oraz w Wydziałach Realizacji urzędów kontroli skarbowej (UKS) i Ministerstwa Finansów przed przekształceniem i przyjęciem statusu funkcjonariusza Służby Celno-Skarbowej nie pełnili służby, a wykonywali pracę, zatem nie można mówić, iż w tym przypadku dochodzi do nierównego traktowania funkcjonariuszy Służby Celno-Skarbowej (wywodzących się, przed konsolidacją, z różnych instytucji). Odnosząc się natomiast do instytucji zmiany formy zatrudnienia, tj. przekształcenia stosunku służbowego w stosunek pracy, należy zauważyć, iż nie tylko w ustawach o Służbie Celnej istniała taka możliwość. Obecnie przepisy ustawy o Krajowej Administracji Skarbowej również pozwalają na przekształcenie stosunku służbowego funkcjonariusza Służby Celno-Skarbowej w stosunek pracy i w zależności od spełnienia przesłanek do nabycia uprawnień mundurowych będą one byłemu funkcjonariuszowi należne lub też nienależne. Ponadto w ustawie o Krajowej Administracji Skarbowej funkcjonują również przepisy dotyczące przekształcenia stosunku pracy członka korpusu służby cywilnej w stosunek służbowy, z zastrzeżeniem, iż członkowi korpusu służby cywilnej, który stał się funkcjonariuszem, okres stażu pracy w służbie cywilnej lub jednostkach organizacyjnych KAS wlicza się do okresu służby w Służbie Celno-Skarbowej, z zastrzeżeniem przepisów ustawy z dnia 18 lutego 1994 r. o zaopatrzeniu emerytalnym funkcjonariuszy Policji, Agencji Bezpieczeństwa Wewnętrznego, Agencji Wywiadu, Służby Kontrwywiadu Wojskowego, Służby Wywiadu Wojskowego, Centralnego Biura Antykorupcyjnego, Straży Granicznej, Służby Ochrony Państwa, Państwowej Straży Pożarnej, Służby Celno-Skarbowej i Służby Więziennej oraz ich rodzin (art. 150 ust. 8 ustawy o KAS). Okresy pracy nie są również traktowane jako okresy służby ani okresy równorzędne ze służbą w rozumieniu ww. ustawy (art. 150 ust. 8a ustawy o KAS). A zatem obecnie w przypadku przeniesienia członka korpusu służby cywilnej do pełnienia służby w Służbie Celno-Skarbowej okres jego pracy mimo zaliczenia stażu pracy do okresu służby, okres pracy nie jest wliczany do uprawnień emerytalnych służb mundurowych. Przedstawiając powyższe wyjaśnienia w zakresie warunków nabycia przez funkcjonariuszy celnych uprawnień do emerytury mundurowej nie jest możliwe zaliczenie do okresu służby, okresów zatrudnienia w korpusie służbie cywilnej. Minister Finansów nie planuje podjąć  prac legislacyjnych w przedmiocie zaliczenia do okresów służby okresów zatrudnienia w służbie cywilnej. Podstawa prawna 1. Ustawa o Rzeczniku Praw Obywatelskich. Z wyrazami szacunku Z upoważnienia Ministra Finansów Sekretarz Stanu Szef Krajowej Administracji Skarbowej Magdalena Rzeczkowska /podpisano kwalifikowanym podpisem elektronicznym/"
df_output.loc[df_output['img_filename'] == 'PDF3.pdf', 'ref_text'] = "RZECZPOSPOLITA POLSKA MINISTER FINANSÓW Warszawa, 8 stycznia 2022 roku KOMUNIKAT MINISTERSTWA FINANSÓW Komunikat z dnia: 8.01.2021 Godziny: 14:00 Od (instytucja zgłaszająca): Ministerstwo Finansów Do (odbiorca): Podmioty wykonujące działalność leczniczą Do wiadomości: - Dotyczy: Zmienione zasady naliczania zaliczek na podatek dochodowy – wynagrodzenie w styczniu Podpisany przez (osoba): Podsekretarz Stanu w MF Jan Sarnowski Priorytet (wstaw znak x): zwykły pilny x Instytucja przekazująca: Narodowy Fundusz Zdrowia telefon e-mail x Sposób przekazania: (wstaw znak x) SMS x SZOI/ PŚw. (NFZ) x komunikator intern. x strona www TREŚĆ KOMUNIKATU Na mocy rozporządzenia Ministra Finansów z 7 stycznia 2022 r. w sprawie przedłużenia terminów poboru i przekazania przez niektórych płatników zaliczek na podatek dochodowy od osób fizycznych, od 8 stycznia 2022 r. zmianie uległy zasady rozliczania zaliczek na podatek dochodowy, pobieranych przez płatników w styczniu i kolejnych miesiącach 2022 r. Zaliczka za styczeń u pracowników czy zleceniobiorców osiągających miesięczny przychód do 12 800 zł brutto, powinna być pobrana w wysokości nie wyższej niż na zasadach działających w zeszłym roku, tj. do 31 grudnia 2021 r. W efekcie, wynagrodzenia wypłacane ww. pracownikom „na rękę” (netto), nie będą niższe niż gdyby były im wypłacane na zasadach z poprzedniego roku. Zaliczki na podatek dochodowy pobierane przez płatników po 7 stycznia 2022 r., powinny być obliczane w sposób opisany w rozporządzeniu. Z związku z czym prosimy płatników o pilne skorygowanie naliczonych wynagrodzeń przed najbliższą wypłatą. Zaliczki na podatek dochodowy pobrane w wyższej wysokości, muszą zostać przeliczone ponownie, a różnica powinna być przez płatnika niezwłocznie zwrócona podatnikowi. Tym samym, pracownicy będą otrzymać korekty od płatników, jeżeli wypłacone wynagrodzenia nie uwzględniały metody pobierania zaliczek na podatek dochodowy, opisanej w rozporządzeniu. Na stronie podatki.gov.pl są umieszczane informacje na temat pobierania zaliczek według nowych zasad, w tym schemat obliczania, materiał szczegółowo wyjaśniający zmiany w rozliczeniu wypłacanych w styczniu wynagrodzeń oraz sposób dokonywania i wypłaty pracownikom ew. korekt. Dodatkowych informacji udzieli (osoba, mail, nr telefonu) www.podatki.gov.pl ul. Świętokrzyska 12   00-916 Warszawa 1/2 tel.: +48 22 694 55 55 • fax: +48 22 694 36 84 • e-mail: kancelaria@mf.gov.pl Z upoważnienia Ministra Finansów Podsekretarz Stanu $Departament/Biuro Jan Sarnowski /podpisano kwalifikowanym podpisem elektronicznym/"
df_output.loc[df_output['img_filename'] == 'RĘCZNY1.png', 'ref_text'] = "Warszawa, 27 lutego 2025 r. URZĄD SKARBOWY WARSZAWA ŚRÓDMIEŚCIE UL. NOWOGRODZKA 3/5 00-513 WARSZAWA JAN KOWALSKI UL. KLONOWA 15/3 00-001 WARSZAWA DOTYCZY: INFORMACJA O ZWROCIE NADPŁATY PODATKU DOCHODOWEGO ZA ROK 2024 Szanowny Panie, Urząd Skarbowy Warszawa Śródmieście informuje, że w wyniku rozliczenia rocznego podatku dochodowego od osób fizycznych za rok 2024 stwierdzono nadpłatę w wysokości 1250,00 zł. Zwrot nadpłaty zostanie dokonany na wskazany w zeznaniu podatkowym rachunek bankowy o numerze 12 3456 7890 1234 5678 9012 3456 w terminie do 30 dni od daty doręczenia niniejszego pisma. W przypadku jakichkolwiek pytań lub niezgodności prosimy o kontakt z urzędem skarbowym pod numerem telefonu: 22 123 4567 lub osobiście w siedzibie urzędu. Z poważaniem, Marek Nowak Naczelnik Urzędu Skarbowego Warszawa Śródmieście"
df_output.loc[df_output['img_filename'] == 'RĘCZNY2.png', 'ref_text'] = "NACZELNIK URZĘDU SKARBOWEGO WROCŁAW-FABRYCZNA Wrocław, 26 lutego 2023 roku UNP: SPRAWA: INFORMACJA O HIPOTETYCZNYM PODATKU NALEŻNYM ZNAK SPRAWY: 0224-7010.6624508.2023 KONTAKT: URZĄD SKARBOWY WROCŁAW-FABRYCZNA ALEKSANDRA OSTROWSKIEGO 5 53-238 WROCŁAW INFORMACJA o zwrocie podatku należnego w kwocie 124,00 zł. Szanowny(a) Panie(i), 1 lipca 2022 r. zostały wprowadzone zmiany w przepisach ustawy o podatku dochodowym od osób fizycznych, w tym została zlikwidowana tzw. ulga dla klasy średniej. Nowe przepisy przewidują jednocześnie zabezpieczenie dla podatników, którzy tracą na likwidacji tej ulgi. Wprowadzono możliwość dokonania zwrotu różnicy podatku należnego w przypadku, gdy przy uwzględnieniu ulgi dla klasy średniej obliczony hipotetyczny podatek należny jest niższy niż wskazany w zeznaniu podatkowym za rok 2022."
df_output.loc[df_output['img_filename'] == 'RĘCZNY3.png', 'ref_text'] = "Warszawa, 27 lutego 2025 r. JAN KOWALSKI UL. KLONOWA 15/3 00-001 WARSZAWA TEL. 600 123 456 URZĄD SKARBOWY WARSZAWA-ŚRÓDMIEŚCIE UL. NOWOGRODZKA 3/5 00-513 WARSZAWA DOTYCZY: PROŚBA O ROZŁOŻENIE ZALEGŁOŚCI PODATKOWEJ NA RATY Szanowni Państwo, Zwracam się z uprzejmą prośbą o rozłożenie na raty należności z tytułu podatku dochodowego za rok 2024 w wysokości 5200 zł. Powodem mojej prośby jest trudna sytuacja finansowa spowodowana utratą pracy oraz niespodziewanymi wydatkami związanymi z leczeniem. W związku z powyższym wnoszę o rozłożenie płatności na 6 równych rat po 867 zł każda. Jestem gotów przedstawić dodatkowe dokumenty potwierdzające moją sytuację, jeśli będzie to konieczne. Z poważaniem, JAN KOWALSKI"

for index, row in df_output.iterrows():
  filename = row['img_filename']
  ref = row['ref_text']
  output = row['ocr_output']
  cer = fastwer.score_sent(output, ref, char_level=True)
  wer = fastwer.score_sent(output, ref, char_level=False)
  df_output.loc[df_output['img_filename'] == filename, 'cer'] = round(cer,2) # Round value to 2 decimal places
  df_output.loc[df_output['img_filename'] == filename, 'wer'] = round(wer,2)

df_output

,img_filename,ocr_output,ref_text,cer,wer
0,PDF1.pdf,"=== Strona 1 ===gia Warszawa, dnia 16 kwietnia...","Warszawa, dnia 16 kwietnia 2021 r. RZECZPOSPOL...",3.46,23.13
1,PDF2.pdf,"=== Strona 1 ===SD s (4, RZECZPOSPOLITA POLSKA...",RZECZPOSPOLITA POLSKA MINISTER FINANSÓW Warsza...,2.55,22.17
2,PDF3.pdf,=== Strona 1 ===DN s Z RZECZPOSPOLITA POLSKASZ...,RZECZPOSPOLITA POLSKA MINISTER FINANSÓW Warsza...,8.86,34.0
3,PNG1.png,"Wrocław, 24.08.2022Jan Kowalskiul Piękna 150-1...","Wrocław, 24.08.2022 Jan Kowalski ul. Piękna 1 ...",5.41,50.0
4,PNG2.png,"Warszawa, dn. 08 siorpnia 2016 rMINISTER OBRON...","Warszawa, dn. 09 sierpnia 2016 r MINISTER OBRO...",17.94,63.26
5,PNG3.jpg,"47, RZECZPOSPOLITA POLSKASS ZA ńZS MINISTERSTW...",RZECZPOSPOLITA POLSKA MINISTERSTWO FINANSÓW SE...,6.57,34.43
6,RĘCZNY1.png,WARSZAWA 2% LUTEGO 202 K.RUD SKARBOWY WARSZA Ś...,"Warszawa, 27 lutego 2025 r. URZĄD SKARBOWY WAR...",66.28,89.92
7,RĘCZNY2.png,Ć NACZELNIK URZĘDUSKARGOLKEGO HROUŁAN”FABRYCZN...,NACZELNIK URZĘDU SKARBOWEGO WROCŁAW-FABRYCZNA ...,66.98,96.26
8,RĘCZNY3.png,"WARSZAWA, Ż7 LUTEGOAN KOWALSKI KOREUL. KLONOWE...","Warszawa, 27 lutego 2025 r. JAN KOWALSKI UL. K...",63.61,92.23
